In [2]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.stats import sigma_clipped_stats
import glob
import os
import subprocess
import warnings
import numpy as np
import matplotlib.pyplot as plt
import photutils
import pyregion

In [48]:
#to know the files
for i in ['data','bias','flat']:
    os.chdir(i)
    c=[x for x in os.listdir() if 'Zone' not in x]
    print(c)
    os.chdir('../')
curpath=os.path.abspath('.') 

['20230521151417-752-RA.fits', '20230522153045-729-RA.fits', '20230524182725-957-RA.fits', '20230526152122-558-RA.fits', '20230523210842-911-RA.fits', '20230521154933-241-RA.fits', '20230520151900-863-RA.fits']
['20230525_masterbias.fits', '20230528_masterbias.fits', '20230603_masterbias.fits', '20230605_masterbias.fits', '20230530_masterbias.fits', '20230604_masterbias.fits']
['20230526_masterflat_g_.fits', '20230527_masterflat_g_.fits', '20230603_masterflat_g_.fits', 'masterflat_g_.fits', '20230606_masterflat_g_.fits', '20230605_masterflat_g_.fits']


In [49]:
#files name lists
data_files=['20230521151417-752-RA.fits', '20230522153045-729-RA.fits', '20230524182725-957-RA.fits', '20230526152122-558-RA.fits', '20230523210842-911-RA.fits', '20230521154933-241-RA.fits', '20230520151900-863-RA.fits']
bias_files=['20230525_masterbias.fits', '20230528_masterbias.fits', '20230603_masterbias.fits', '20230605_masterbias.fits', '20230530_masterbias.fits', '20230604_masterbias.fits']
flat_files=['20230526_masterflat_g_.fits', '20230527_masterflat_g_.fits', '20230603_masterflat_g_.fits','20230606_masterflat_g_.fits', '20230605_masterflat_g_.fits']
#ignoring the general masterflat as using closest date

In [42]:
#creating proc folder
procFolder = os.path.join(curpath, 'processing')

if not os.path.isdir(procFolder): 
    os.mkdir(procFolder)
else:
    for f in os.listdir(procFolder):
        try:
            os.remove(os.path.join(procFolder,f)) # clear the processing folder from previous iterations
        except:
            print('Could not remove',f)

procList = [os.path.join(procFolder, file).replace('.fits','.proc.fits') for file in data_files]


In [1]:
#usefull definitions

def find_closest(data_date, calibfiles):
    calib_file_dates = [(file, extract_date(file)) for file in calibfiles]
    closest_file = None
    min_diff=np.inf
    for file, date in calib_file_dates:
        date_diff = abs(date - data_date)
        if date_diff < min_diff:
            min_diff = date_diff
            closest_file = file    
    return closest_file

def extract_date(filename):
    return int(filename[:8])

In [47]:
#processing
for data_file in data_files:
    data_date = extract_date(data_file)
    closest_bias = find_closest(data_date, bias_files)
    closest_flat = find_closest(data_date, flat_files)

    HDUList = fits.open(os.path.join(curpath, 'data', data_file))
    primaryHeader = HDUList[0].header
    if len(HDUList) > 1: #as got error - maybe header missing in some files
        imageData = HDUList[1].data
    else:
        imageData = HDUList[0].data
    HDUList.close()

    HDUList = fits.open(os.path.join(curpath, 'bias', closest_bias))
    biasData = HDUList[0].data    
    HDUList.close()

    procData = (imageData - biasData)

    HDUList = fits.open(os.path.join(curpath, 'flat', closest_flat))
    flatData = HDUList[0].data    
    HDUList.close()

    procData = procData / flatData

    procHDU = fits.PrimaryHDU(procData)
    procHDU.header = primaryHeader
    procHDU.header.add_history('Bias corrected and flat-fielded')
    output_filename = procList[data_files.index(data_file)]
    procHDU.writeto(output_filename, overwrite=True)
    print(f'{data_file} -> {output_filename}')



/tmp/ipykernel_2804/3303922994.py:29: RuntimeWarning: divide by zero encountered in divide
  procData = procData / flatData


20230521151417-752-RA.fits -> /home/mayank/astronomybro/g_band/processing/20230521151417-752-RA.proc.fits


/tmp/ipykernel_2804/3303922994.py:29: RuntimeWarning: invalid value encountered in divide
  procData = procData / flatData


20230522153045-729-RA.fits -> /home/mayank/astronomybro/g_band/processing/20230522153045-729-RA.proc.fits
20230524182725-957-RA.fits -> /home/mayank/astronomybro/g_band/processing/20230524182725-957-RA.proc.fits
20230526152122-558-RA.fits -> /home/mayank/astronomybro/g_band/processing/20230526152122-558-RA.proc.fits
20230523210842-911-RA.fits -> /home/mayank/astronomybro/g_band/processing/20230523210842-911-RA.proc.fits
20230521154933-241-RA.fits -> /home/mayank/astronomybro/g_band/processing/20230521154933-241-RA.proc.fits
20230520151900-863-RA.fits -> /home/mayank/astronomybro/g_band/processing/20230520151900-863-RA.proc.fits
